In [112]:
INPUT_FILE = "example-1.txt"

In [113]:
INPUT_FILE = "example-2.txt"

In [114]:
INPUT_FILE = "input.txt"

In [115]:
with open(INPUT_FILE, "r") as file:
  fileLines = list(filter(lambda x: len(x) != 0, file.read().split('\n')))
  input = []
  for line in fileLines:
    [signalPatters, digitOutputValues] = list(map(lambda x: x.strip(), line.split("|")))
    input.append({
      'signalPatterns': signalPatters.split(" "),
      'digitOutputValues': digitOutputValues.split(" ")
    })

In [116]:
def countDigitsInLine(length, line):
  return len(list(filter(lambda x: len(x) == length, line['digitOutputValues'])))

def countDigits(length):
  return sum(map(lambda line: countDigitsInLine(length, line), input))

ones = countDigits(2)
fours = countDigits(4)
sevens = countDigits(3)
eights = countDigits(7)


print(sum((ones, fours, sevens, eights)))

237


In [117]:
def getMappingFromLength(length, line):
  result = list(filter(lambda x: len(x) == length, line['signalPatterns']))
  return result

def normalizeMapping(value):
  segments = []
  segments[:0] = value
  segments.sort()
  result = ""
  for segment in segments:
    result += segment
  return result

def getHorizontalSegments(mappings):
  horizontalSegments = None
  for segmentMapping in mappings:
    segments = []
    segments[:0] = segmentMapping
    if horizontalSegments == None:
      horizontalSegments = set(segments)
    else:
      horizontalSegments = horizontalSegments.intersection(segments)
  return horizontalSegments

def getMappingForLine(line):
  mapping = {}

  mapping[1] = getMappingFromLength(2, line)[0]
  mapping[4] = getMappingFromLength(4, line)[0]
  mapping[7] = getMappingFromLength(3, line)[0]
  mapping[8] = getMappingFromLength(7, line)[0]

  # 2,3,5 have a length of 5
  mappingsOfLength5 = getMappingFromLength(5, line)

  # 0,6,9 have a length of 6
  mappingsOfLength6 = getMappingFromLength(6, line)

  # 3 if the only 5 length that has both segments found in 1
  mapping[3] = list(filter(lambda x: mapping[1][0] in x and mapping[1][1] in x,mappingsOfLength5))[0]

  # the shared segments between 2 and 5 will be the horizontal segments
  horizontalSegments = getHorizontalSegments(filter(lambda x: x != mapping[3], mappingsOfLength5))

  # 0 is the only 6 length that is missing a horizontal segments
  mapping[0] = list(filter(lambda x: not all(map(lambda segment: segment in x, horizontalSegments)), mappingsOfLength6))[0]

  # 6 is missing a segment that is present in 4 and 9 is not
  mappingsFor6And9 = list(filter(lambda x: x != mapping[0], mappingsOfLength6))
  segmentsIn4 = []
  segmentsIn4[:0] = mapping[4]
  mapping[6] = list(filter(lambda x: not all(map(lambda segment: segment in x, segmentsIn4)), mappingsFor6And9))[0]
  mapping[9] = list(filter(lambda x: x != mapping[6], mappingsFor6And9))[0]

  # segments missing from 6 and 9 combined are the segments missing from 5
  allSegments = []
  allSegments[:0] = mapping[8] # 8 has all segments
  missingFrom5 = []
  missingFrom5.append(list(filter(lambda x: x not in mapping[6], allSegments))[0])
  missingFrom5.append(list(filter(lambda x: x not in mapping[9], allSegments))[0])
  mappingsFor2and5 = list(filter(lambda x: x != mapping[3], mappingsOfLength5))
  mapping[5] = list(filter(lambda x: all(map(lambda segment: segment not in x, missingFrom5)), mappingsFor2and5))[0]
  mapping[2] = list(filter(lambda x: x != mapping[5], mappingsFor2and5))[0]

  result = {}
  for key, item in mapping.items():
    result[normalizeMapping(item)] = key

  return result

def decodeOutputValue(outputValue, mapping):
  result = 0
  for index, digit in enumerate(outputValue):
    result += mapping[normalizeMapping(digit)] * (10 ** (len(outputValue) - index - 1))
  
  return result

result = sum(map(lambda line: decodeOutputValue(line['digitOutputValues'], getMappingForLine(line)), input))
print(result)

1009098
